# 학술지 전처리

### 1) 폴더 안에 있는 20개의 pdf upstage 이용해서 전처리 후 json으로 저장

### 이후 전처리는 각자 해보고 금욜날 공유 ㄱㄱㄱ

In [2]:
from typing import TypedDict, Annotated, List, Dict
import operator

class GraphState(TypedDict):
    folderpath : Annotated[str, "filepath"]  # 원본 파일 경로
    
    analyzed_files : Annotated[List, "analyzed_files"]

    metadata: Annotated[List[Dict], operator.add]  # parsing metadata (api, model, usage)
    
    elements_from_parser: Annotated[List[Dict], "elements_from_parser"]  

state = GraphState(folderpath='/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지')
state

{'folderpath': '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지'}

In [14]:
import requests
import json
import os

DEFAULT_CONFIG = {
    "ocr": True,
    "coordinates": True,
    "output_formats": "['html', 'text', 'markdown']",
    "model": "document-parse",
    "base64_encoding": "['figure', 'chart', 'table']",
}

def analyze_layout(state: GraphState):
    """
    분할된 PDF 파일들을 Upstage API로 전송하여 레이아웃 분석을 수행하고,
    결과 JSON 파일들을 저장하는 함수.

    Returns:
    - list: 성공적으로 분석된 파일들의 결과 JSON 파일 경로 목록.
    """

    # '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지/Apm006-01-17.pdf'

    docs_folder = '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지'    # <- 20개 pdf 파일이 있는 폴더 경로

    output_folder = "/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage"      # <- 최종 upstage 결과 저장 폴더
    
    api_url = "https://api.upstage.ai/v1/document-ai/document-parse"
    api_key = os.environ.get("UPSTAGE_API_KEY")

    # split_docs 폴더 내의 모든 PDF 파일 가져오기
    pdf_files = [f for f in os.listdir(docs_folder) if f.endswith(".pdf")]
    print(pdf_files)
    output_paths = []
    metadata_list = []
    
    for pdf_file in pdf_files:

        if pdf_file == 'Apm006-01-17.pdf':

            file_path = os.path.join(docs_folder, pdf_file)
            output_json_path = os.path.join(output_folder, f"{os.path.splitext(pdf_file)[0]}.json")
            print(f"📤 파일 업로드 중: {pdf_file}")
            
            with open(file_path, "rb") as pdf:
                response = requests.post(
                    api_url,
                    headers={"Authorization": f"Bearer {api_key}"},
                    data=DEFAULT_CONFIG,
                    files={"document": pdf}
                )
            
            # 응답 확인 및 JSON 파일 저장
            if response.status_code == 200:
                result = response.json()
                with open(output_json_path, "w", encoding="utf-8") as json_file:
                    json.dump(result, json_file, ensure_ascii=False, indent=4)
                print(f"✅ 분석 결과 저장 완료: {output_json_path}")
                output_paths.append(output_json_path)
                meta = {
                    "id": pdf_file,
                    "model": result.get("model"),
                    "usage": result.get("usage")
                }
                metadata_list.append(meta)
            else:
                print(f"❌ 오류 발생 ({response.status_code}): {response.text}")
        
        print("🎉 모든 파일 분석 완료!")
        return GraphState(metadata=metadata_list,
                        analyzed_files=output_paths)

In [15]:
state_out = analyze_layout(state)
state.update(state_out)
state

['Apm006-03-19.pdf', 'Apm005-04-18.pdf', 'Apm005-04-19.pdf', 'Apm006-03-18.pdf', 'Apm006-03-20.pdf', 'Apm006-01-18.pdf', 'Apm005-03-17.pdf', 'Apm006-02-19.pdf', 'Apm005-02-16.pdf', 'Apm006-02-21.pdf', 'Apm005-02-01.pdf', 'Apm005-02-15.pdf', 'Apm004-04-17.pdf', 'Apm004-04-16.pdf', 'Apm006-02-20.pdf', 'Apm006-01-17.pdf', 'Apm005-03-18.pdf', 'Apm006-04-20.pdf', 'Apm005-01-19.pdf', 'Apm005-04-17.pdf']
🎉 모든 파일 분석 완료!


{'folderpath': '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지',
 'metadata': [],
 'analyzed_files': []}

In [18]:
from dotenv import load_dotenv
load_dotenv()

def analyze_layout(state: GraphState):

    output_folder = "/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage"
    output_json_path = os.path.join(output_folder, f"Apm006-01-17.json")
    
    api_url = "https://api.upstage.ai/v1/document-ai/document-parse"
    api_key = os.environ.get("UPSTAGE_API_KEY")

    file_path = '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지/Apm006-01-17.pdf'

    output_paths = []
    metadata_list = []
    
    with open(file_path, "rb") as pdf:
        response = requests.post(
            api_url,
            headers={"Authorization": f"Bearer {api_key}"},
            data=DEFAULT_CONFIG,
            files={"document": pdf}
        )

    if response.status_code == 200:
        result = response.json()
        with open(output_json_path, "w", encoding="utf-8") as json_file:
            json.dump(result, json_file, ensure_ascii=False, indent=4)
        print(f"✅ 분석 결과 저장 완료: {output_json_path}")
        output_paths.append(output_json_path)
        metadata_list.append({
            "id": 'Apm006-01-17',
            "model": result.get("model"),
            "usage": result.get("usage")
        })
    else:
        print(f"❌ 오류 발생 ({response.status_code}): {response.text}")
    
    print("🎉 파일 분석 완료!")
    return GraphState(metadata=metadata_list, analyzed_files=output_paths)

In [19]:
state_out = analyze_layout(state)
state.update(state_out)
state

✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm006-01-17.json
🎉 파일 분석 완료!


{'folderpath': '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지',
 'metadata': [{'id': 'Apm006-01-17',
   'model': 'document-parse-250404',
   'usage': {'pages': 4}}],
 'analyzed_files': ['/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm006-01-17.json']}

# 노가다 전처리 끝난 데이터로 질문 생성

In [ ]:
from openai import OpenAI
import json
from tqdm import tqdm
import os
from dotenv import load_dotenv

system_prompt = """
# Role and Objective
You are a specialized medical expert in Pediatric Anesthesia.
Your primary task is to generate accurate and clinically relevant Korean Q&A data based on pediatric anesthesia literature, including case reports, clinical research, and academic publications, to support training a medical question-answering model.

# Core Principles
- Generate exactly **10** high-quality Q&A pairs
- Maintain strict medical accuracy and precision
- Focus on practical clinical scenarios
- Ensure questions are specific and actionable
- Include relevant numerical values when available
- Generate questions only from provided context
- Use clear and professional Korean language

# Question Generation Guidelines
1. Content Adherence:
   - Generate questions strictly based on provided content
   - Include specific numerical values (doses, weights, ages, etc.)
   - Focus on practical clinical applications

2. Question Format:
   - Use open-ended, short-answer format
   - Write questions in Korean
   - Make questions specific and clinically relevant
   - Avoid multiple-choice formats

3. Reference Requirements:
   - Include all supporting sentences
   - For tables: include entire table content
   - For figures: include surrounding text and captions
4. Avoid generating questions about study design, patient recruitment, or statistical methods unless they are clinically significant.
5. Prioritize questions focusing on diagnosis, treatment, medication usage, risk factors, and the practical application of research findings.
6. Avoid questions that merely ask for specific study results or numerical comparisons (e.g., changes in blood pressure or differences between groups). Instead, convert them into questions that explore the clinical implications or applications of these findings.

# Answer Guidelines
- Start reasoning directly with a quote (no phrases like '문서에 따르면', '문서에 의하면')
- Start with a step-by-step reasoning using quotes from the context
- Enclose all direct quotes in **##begin_quote## ... ##end_quote##**
- End with `<ANSWER>: ...` in Korean (full sentence) 

# Output Format
Return your output in this **strict JSON format**:
[
  {
    "question": "한국어로 된 의학 질문",
    "reference_sentences": [
      "관련 문장 1",
      "관련 문장 2"
    ],
    "answer": "##Reason: ...\n<ANSWER>: ..."
  }
]

# Quality Controls
- Ensure medical accuracy
- Verify all numerical values
- Check Korean language usage
- Validate JSON format
- Confirm reference alignment

# Error Prevention
- Double-check medical terminology
- Verify numerical calculations
- Ensure proper Korean grammar
- Validate JSON structure
- Confirm context alignment

# Example Outputs
Return as a **valid JSON array** of objects. Each object must look like:

[
  {{
    "question": "5살 23kg 소아 환자의 마취 중 적절한 수액 주입량은?",
    "reference_sentences": [
      "20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다.",
      "이 수액량은 전신마취 중 적절한 수분 공급을 위해 사용된다."
    ],
    "answer": "##Reason: 문서의 ##begin_quote## 20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다 ##end_quote## 라는 설명에 따르면, 23kg 소아는 1500mL + (3×20mL) = 1560mL가 필요합니다.\n<ANSWER>: 5살 23kg 소아의 마취 중 유지 수액량은 1560mL입니다."
  }},
  {{
    "question": "소아 환자를 깨울 때 laryngospasm이 의심되면 어떤 처치를 해야 하나요?",
    "reference_sentences": [
      "Laryngospasm이 의심되는 경우 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다.",
      "신속한 인식과 처치가 저산소증을 예방하는 데 중요하다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다 ##end_quote## 라고 되어 있습니다. 이는 환자의 기도를 유지하고 저산소증을 방지하는 데 중요한 조치입니다.\n<ANSWER>: Laryngospasm이 의심되는 경우 jaw thrust, 양압 환기, succinylcholine 투여를 포함한 즉각적인 처치가 필요합니다."
  }}
]
"""

In [37]:
import os

md_file_path = '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/self/Apm006-03-18.md'

# 파일 읽어서 변수에 저장
with open(md_file_path, 'r', encoding='utf-8') as f:
    data = f.read()
    
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

try:
    response = client.chat.completions.create(
        model="gpt-4.1",  # 또는 gpt-4.0, gpt-3.5-turbo
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": data}
        ],
        temperature=0.5
    )
    
    output = response.choices[0].message.content
    parsed = json.loads(output,strict=False)
    print(parsed)

except Exception as e:
    print(f"❌ Error generating Q&A: {e}")

# # 6. 결과 출력
# print(json.dumps(results, indent=2, ensure_ascii=False))s

[{'question': '편도절제술을 받는 3세에서 10세 소아에서 desflurane 마취 시 각성흥분을 예방하기 위해 마취유도 직전에 투여할 fentanyl의 적정 용량은 얼마인가요?', 'reference_sentences': ['대상 환아를 무작위로 3군으로 나누어 마취유도 직전에 fentanyl 1 �g/kg (1군, n = 26), 2 �g/kg (2군, n = 27), 3 �g/kg (3군, n = 28)을 각각 정주하였다.', '각성흥분의 발생빈도는 3군이 1군보다 유의하게 낮았고(P < 0.05), 3군과 2군 간에는 차이가 없었다(Table 4).', '결론적으로 편도절제술을 시행받는 환아에서 desflurane 마취 시 각성 흥분을 예방하기 위하여 마취유도시 정주한 fentanyl 3 �g/kg는 1 �g/kg의 용량에 비하여 효과적으로 각성흥분을 예방할 수 있었으나, 2 �g/kg의 용량과 비교하여서는 통계학적인 차이가 없었다.'], 'answer': '##Reason: ##begin_quote## 결론적으로 편도절제술을 시행받는 환아에서 desflurane 마취 시 각성 흥분을 예방하기 위하여 마취유도시 정주한 fentanyl 3 �g/kg는 1 �g/kg의 용량에 비하여 효과적으로 각성흥분을 예방할 수 있었으나, 2 �g/kg의 용량과 비교하여서는 통계학적인 차이가 없었다. ##end_quote## 라는 설명을 통해, fentanyl 3 μg/kg이 각성흥분 예방에 효과적임을 알 수 있습니다. 그러나 2 μg/kg과의 차이는 통계적으로 유의하지 않았으므로, 두 용량 모두 임상적으로 고려될 수 있습니다. <ANSWER>: 편도절제술을 받는 소아에서 desflurane 마취 시 각성흥분 예방을 위해 마취유도 직전에 fentanyl 3 μg/kg 또는 2 μg/kg을 투여하는 것이 효과적입니다.'}, {'question': '편도절제술을 받는 소아에서 마취유도 시 fentanyl 3 μg/kg 투여 시 각성 후 심한 통증의 발생률은 어떻게

# Final 생성

In [43]:
system_prompt = """
# Role
You are a board-certified pediatric anesthesiologist tasked with generating medically accurate and clinically meaningful Korean Q&A data from pediatric anesthesia documents.

# Objective
Generate a high-quality QA dataset to fine-tune a Korean-language LLM specialized in pediatric anesthesia. The model will be used by clinical professionals, so questions must reflect what experienced physicians would realistically ask in real-world pediatric anesthesia settings.

# Core Principles
- Generate exactly **10** high-quality Q&A pairs
- Maintain strict medical accuracy and clinical validity
- Focus on practical and realistic clinical scenarios
- Ensure questions are specific, actionable, and non-redundant
- Include relevant numerical values when applicable
- Use professional and natural Korean suitable for clinical communication
- Generate questions **strictly based on the provided context**

# Question Generation Guidelines
1. **Clinical Relevance**
   - Focus on: diagnosis, treatment decisions, anesthesia drug usage, risk management, intra/post-op complications, airway management, fluid/blood dosing, sedation recovery
   - Avoid: basic memorization, definitions, or simple score lookups
   - Avoid questions derived from **tables** (e.g., “Ramsey sedation score 3점은 무엇인가요?”)
   - Avoid referencing the specific case (e.g., “본 연구에서”, “본 증례에서”) — all questions must be **generalizable**

2. **Non-Redundancy**
   - Do not include similar or overlapping questions
   - If multiple potential questions exist about the same topic, select only the **most clinically useful one**

3. **Question Format**
   - Open-ended, short-answer format
   - Written in natural, professional Korean
   - No multiple-choice or yes/no questions
   - Avoid awkward or repetitive phrasing

4. **Reference Requirements**
   - Include relevant support sentences as `"reference_sentences"` array
   - For figures: include captions and surrounding explanations
   - **Do not use content that appears only in tables**

5. **Answer Format**
   - Begin with a reasoning step using evidence from the context
   - Wrap quoted evidence in **##begin_quote## ... ##end_quote##**
   - End with a Korean full-sentence answer prefixed by `<ANSWER>:`

# Output Format
Return your output in this **strict JSON format**:
[
  {
    "question": "한국어로 된 의학 질문",
    "reference_sentences": [
      "관련 문장 1",
      "관련 문장 2"
    ],
    "answer": "##Reason: ...\n<ANSWER>: ..."
  }
]

# Quality Checks
- Verify all medical terminology and numerical values
- Check Korean grammar and phrasing
- Ensure JSON format validity
- Eliminate redundant or low-value questions

# Error Prevention
- Double-check medical terminology
- Verify numerical calculations
- Ensure proper Korean grammar
- Validate JSON structure
- Confirm context alignment

# Example Outputs
Return as a **valid JSON array** of objects. Each object must look like:

[
  {{
    "question": "5살 23kg 소아 환자의 마취 중 적절한 수액 주입량은?",
    "reference_sentences": [
      "20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다.",
      "이 수액량은 전신마취 중 적절한 수분 공급을 위해 사용된다."
    ],
    "answer": "##Reason: 문서의 ##begin_quote## 20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다 ##end_quote## 라는 설명에 따르면, 23kg 소아는 1500mL + (3×20mL) = 1560mL가 필요합니다.\n<ANSWER>: 5살 23kg 소아의 마취 중 유지 수액량은 1560mL입니다."
  }},
  {{
    "question": "소아 환자를 깨울 때 laryngospasm이 의심되면 어떤 처치를 해야 하나요?",
    "reference_sentences": [
      "Laryngospasm이 의심되는 경우 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다.",
      "신속한 인식과 처치가 저산소증을 예방하는 데 중요하다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다 ##end_quote## 라고 되어 있습니다. 이는 환자의 기도를 유지하고 저산소증을 방지하는 데 중요한 조치입니다.\n<ANSWER>: Laryngospasm이 의심되는 경우 jaw thrust, 양압 환기, succinylcholine 투여를 포함한 즉각적인 처치가 필요합니다."
  }}
]
"""

In [45]:
system_prompt_1 = """
# Role and Objective
You are a specialized medical expert in Pediatric Anesthesia.
Your primary task is to generate accurate and clinically relevant Korean Q&A data based on pediatric anesthesia literature, including case reports, clinical research, and academic publications, to support training a medical question-answering model.

# Core Principles
- Generate exactly **10** high-quality Q&A pairs
- Maintain strict medical accuracy and precision
- Focus on practical clinical scenarios
- Ensure questions are specific and actionable
- Include relevant numerical values when available
- Generate questions only from provided context
- Use clear and professional Korean language

# Question Generation Guidelines
1. Content Adherence:
   - Generate questions strictly based on provided content
   - Include specific numerical values (doses, weights, ages, etc.)
   - Focus on practical clinical applications

2. Question Format:
   - Use open-ended, short-answer format
   - Write questions in Korean
   - Make questions specific and clinically relevant
   - Avoid multiple-choice formats

3. Reference Requirements:
   - Include all supporting sentences
   - For tables: include entire table content
   - For figures: include surrounding text and captions
4. Avoid generating questions about study design, patient recruitment, or statistical methods unless they are clinically significant.
5. Prioritize questions focusing on diagnosis, treatment, medication usage, risk factors, and the practical application of research findings.
6. Avoid questions that merely ask for specific study results or numerical comparisons (e.g., changes in blood pressure or differences between groups). Instead, convert them into questions that explore the clinical implications or applications of these findings.

# Answer Guidelines
- Start reasoning directly with a quote (no phrases like '문서에 따르면', '문서에 의하면')
- Start with a step-by-step reasoning using quotes from the context
- Enclose all direct quotes in **##begin_quote## ... ##end_quote##**
- End with `<ANSWER>: ...` in Korean (full sentence) 

# Output Format
Return your output in this **strict JSON format**:
[
  {
    "question": "한국어로 된 의학 질문",
    "reference_sentences": [
      "관련 문장 1",
      "관련 문장 2"
    ],
    "answer": "##Reason: ...\n<ANSWER>: ..."
  }
]

# Quality Controls
- Ensure medical accuracy
- Verify all numerical values
- Check Korean language usage
- Validate JSON format
- Confirm reference alignment

# Error Prevention
- Double-check medical terminology
- Verify numerical calculations
- Ensure proper Korean grammar
- Validate JSON structure
- Confirm context alignment

# Example Outputs
Return as a **valid JSON array** of objects. Each object must look like:

[
  {{
    "question": "5살 23kg 소아 환자의 마취 중 적절한 수액 주입량은?",
    "reference_sentences": [
      "20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다.",
      "이 수액량은 전신마취 중 적절한 수분 공급을 위해 사용된다."
    ],
    "answer": "##Reason: 문서의 ##begin_quote## 20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다 ##end_quote## 라는 설명에 따르면, 23kg 소아는 1500mL + (3×20mL) = 1560mL가 필요합니다.\n<ANSWER>: 5살 23kg 소아의 마취 중 유지 수액량은 1560mL입니다."
  }},
  {{
    "question": "소아 환자를 깨울 때 laryngospasm이 의심되면 어떤 처치를 해야 하나요?",
    "reference_sentences": [
      "Laryngospasm이 의심되는 경우 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다.",
      "신속한 인식과 처치가 저산소증을 예방하는 데 중요하다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다 ##end_quote## 라고 되어 있습니다. 이는 환자의 기도를 유지하고 저산소증을 방지하는 데 중요한 조치입니다.\n<ANSWER>: Laryngospasm이 의심되는 경우 jaw thrust, 양압 환기, succinylcholine 투여를 포함한 즉각적인 처치가 필요합니다."
  }}
]
"""

In [46]:
from tqdm import tqdm
import os
from dotenv import load_dotenv
import csv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

folder_path = '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/self'

output_csv_path = "학술지_QnA_1.csv"
csv_rows = []

for file_name in tqdm(os.listdir(folder_path)):

    file_path = os.path.join(folder_path, file_name)

    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.read()

    try:
        response = client.chat.completions.create(
            model="gpt-4.1",  # 또는 gpt-4.0, gpt-3.5-turbo
            messages=[
                {"role": "system", "content": system_prompt_1},
                {"role": "user", "content": data}
            ],
            temperature=0.2
        )
        
        output = response.choices[0].message.content.strip()
        qna_list = json.loads(output,strict=False)

        for qna in qna_list:
            question = qna['question']
            reference_sentences = qna['reference_sentences']
            answer = qna['answer']
            csv_rows.append([file_name, question, reference_sentences, answer])

    except Exception as e:
        print(f"❌ Error processing {file_name}: {e}")
        continue

# 5. 최종 CSV 저장
with open(output_csv_path, "w", newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["파일명", "질문", "관련 문서", "답변"])  # 헤더
    writer.writerows(csv_rows)

print(f"✅ CSV 저장 완료: {output_csv_path}")


  0%|          | 0/19 [00:00<?, ?it/s]

100%|██████████| 19/19 [13:06<00:00, 41.37s/it]

✅ CSV 저장 완료: 학술지_QnA_1.csv
